In [30]:
import os

In [2]:
%pwd

'c:\\Users\\abhay\\Desktop\\Internship\\Data-Engineering-Internship-Bosch\\notebook'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\abhay\\Desktop\\Internship\\Data-Engineering-Internship-Bosch'

In [31]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
     

In [32]:
from src.dataEngineer.utils.common import create_directories, read_yaml
CONFIG_FILE_PATH = Path("config/config.yaml")

In [33]:
class ConfigurationManager:
    def __init__(self):
        self.config = read_yaml(CONFIG_FILE_PATH)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )

        return data_ingestion_config

        


In [34]:
import os
import urllib.request as request
import zipfile
from src.dataEngineer import logger
from src.dataEngineer.utils.common import *

In [35]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")

        else:
            logger.info(f"file already exist")


    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        try:
            with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
        except Exception as e:
            logger.info("File is not a zip file")

In [36]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2025-08-14 21:47:45,846: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-14 21:47:45,856: INFO: common: created directory at : artifacts]
[2025-08-14 21:47:45,857: INFO: common: created directory at : artifacts/data_ingestion]
[2025-08-14 21:47:45,859: INFO: 1585119331: file already exist]
